```{contents}
```
## Custom Tools

### What Custom Tools Are

**Custom tools** are **user-defined functions** that you expose to an LLM/agent so it can **perform deterministic actions** such as:

* Querying databases
* Calling internal APIs
* Running business logic
* Reading files or metrics

> Custom tools let the LLM **decide *when* to act**, while your code **controls *how* the action runs**.

---

### Why Custom Tools Are Needed

LLMs alone cannot:

* Access live systems
* Perform reliable calculations
* Enforce business rules
* Execute side effects safely

Custom tools enable:

* Deterministic behavior
* Enterprise integration
* Auditable actions
* Safer agent workflows

---

### Where Custom Tools Fit

```
User Input
   ↓
LLM / Agent (reasoning)
   ↓
Custom Tool (your code)
   ↓
Tool Result
   ↓
LLM Final Answer
```

---

### What Makes a Tool “Custom”

A custom tool is:

* Written by **you**
* Has a **clear purpose**
* Uses a **strict input schema**
* Returns a **deterministic output**

---

### Tool vs Function vs API

| Concept     | Meaning                         |
| ----------- | ------------------------------- |
| Function    | Python function                 |
| Tool        | Function + schema + description |
| API         | Remote service                  |
| Custom Tool | Your function exposed as a tool |

LangChain turns **functions into tools**.

---

### How LangChain Understands a Tool

Each tool has:

1. **Name**
2. **Description** (critical for tool selection)
3. **Input schema**
4. **Return value**

The LLM sees **only the schema and description**, not your code.

---

### Method 1: Custom Tool Using `@tool` Decorator (Recommended)

#### Step 1: Define the Tool



In [1]:

from langchain_classic.tools import tool

@tool
def ticket_count(source: str) -> int:
    """Return total number of tickets for a given source (e.g., jira, servicenow)."""
    if source.lower() == "jira":
        return 128
    return 0




What LangChain derives automatically:

* Tool name → `ticket_count`
* Input schema → `{ source: string }`
* Output type → `int`

---

#### Step 2: Bind Tool to LLM



In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

llm_with_tools = llm.bind_tools([ticket_count])



---

#### Step 3: Invoke

In [4]:
# First invoke - LLM decides to call the tool
response = llm_with_tools.invoke(
    "How many tickets are there in Jira?"
)

print("Response from LLM:")
print(response)
print("\nTool calls:", response.tool_calls)

# Execute the tool manually
from langchain_core.messages import ToolMessage

tool_call = response.tool_calls[0]
tool_result = ticket_count.invoke(tool_call["args"])

print(f"\nTool result: {tool_result}")

# Send tool result back to LLM for final answer
messages = [
    ("human", "How many tickets are there in Jira?"),
    response,
    ToolMessage(
        content=str(tool_result),
        tool_call_id=tool_call["id"]
    )
]

final_response = llm_with_tools.invoke(messages)
print(f"\nFinal answer: {final_response.content}")


Response from LLM:
content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 64, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ee69c2ef48', 'id': 'chatcmpl-CpcKpxqkbNoiSx3kZs9kN9e6zmdnI', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b46b9-687b-7f83-a7e9-33922d7488a4-0' tool_calls=[{'name': 'ticket_count', 'args': {'source': 'jira'}, 'id': 'call_OZotasFbqQWbdJQ56TmvZFuI', 'type': 'tool_call'}] usage_metadata={'input_tokens': 64, 'output_tokens': 14, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

Tool calls: [{'name': 'ticket_co

---

### Easier Approach: Use an Agent

Instead of manually handling tool calls, use an agent:


In [5]:
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# Create agent prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools when needed."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# Create agent
agent = create_tool_calling_agent(llm, [ticket_count], prompt)

# Create executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[ticket_count],
    verbose=True
)

# Now just invoke - tool execution is automatic!
result = agent_executor.invoke({
    "input": "How many tickets are there in Jira?"
})

print(f"\nFinal output: {result['output']}")





> Entering new AgentExecutor chain...

Invoking: `ticket_count` with `{'source': 'jira'}`


128There are 128 tickets in Jira.

> Finished chain.

Final output: There are 128 tickets in Jira.


**Key Difference:**

* `bind_tools()` → LLM requests tool calls (you handle execution)
* `AgentExecutor` → Automatic tool execution loop

